# Estival/Nevergrad

Estival wraps summer2 models along with associated data and parameter information, for use in a variety of calibration and optimization contexts

In [ ]:
# Uncomment to install in colab
#!pip install estival==0.3.0

In [ ]:
import summer2
import numpy as np
import pandas as pd

In [ ]:
from summer2.extras import test_models

In [ ]:
m = test_models.sir()
defp = m.get_default_parameters()
defp

In [ ]:
m.run()
m.get_outputs_df().plot();

In [ ]:
# Now let's construct some noisy data to fit to

target_params = {"contact_rate": 0.5, "recovery_rate": 0.4}

m.run(target_params)
do_def = m.get_derived_outputs_df()
obs_clean = do_def["incidence"].iloc[0:50]
obs_noisy = obs_clean * np.exp(np.random.normal(0.0,0.2,len(obs_clean)))
obs_clean.plot()
obs_noisy.plot(style='.');

In [ ]:
# The following imports are the 'building blocks' of estival models

# Targets represent data we are trying to fit to
from estival import targets as est

# We specify parameters using (Bayesian) priors
from estival import priors as esp

# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
from estival.model import BayesianCompartmentalModel

# Nevergrad

In this example, we will be using nevergrad - a gradient free optimization toolkit

https://facebookresearch.github.io/nevergrad/

Estival provides simple wrappers to use BayesianCompartmentalModel objects with a number of optimizers



In [ ]:
# Import nevergrad
import nevergrad as ng

# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model

In [ ]:
# Specify a Truncated normal target with a free dispersion parameter,
# using the noisy data we constructed earlier
targets = [
    est.TruncatedNormalTarget("incidence", obs_noisy, (0.0,np.inf),
        esp.UniformPrior("incidence_dispersion",(0.1, obs_noisy.max()*0.1)))
]

In [ ]:
# Priors over our 2 model parameters
priors = [
    esp.UniformPrior("contact_rate", (0.01,1.0)),
    esp.TruncNormalPrior("recovery_rate", 0.5, 0.2, (0.01,1.0)),
]

In [ ]:
# The BayesianCompartmentalModel class is the primary entry point to all optimization and calibration
# methods in estival
# It takes a CompartmentalModel object, default parameters, priors, and targets
# The default parameters will be used as fixed values when no prior is specified for a given parameter

bcm = BayesianCompartmentalModel(m, m.get_default_parameters(), priors, targets)

In [ ]:
# The resulting object contains all priors, including dispersion parameters, nested values etc
bcm.priors

In [ ]:
# ... and a loglikelihood evaluation function
# This function requires values for all the bcm.priors - not just the CompartmentalModel parameters

ok_params = {"contact_rate": 0.3, "recovery_rate": 0.2, "incidence_dispersion": 1.0}

print(bcm.loglikelihood(**ok_params))

bad_params = {"contact_rate": 0.9, "recovery_rate": 0.6, "incidence_dispersion": 1.0}

print(bcm.loglikelihood(**bad_params))

In [ ]:
# It also provides a convenience method for running parameters against the specified model

ax = bcm.run(ok_params).derived_outputs["incidence"].plot(label="good")
bcm.run(bad_params).derived_outputs["incidence"].plot(label="bad")
ax.legend();

# Using the optimizers

optimize_model is the main entry point to using nevergrad optimizers.<br>
It wraps the loglikelihood function for minimization (0.0 - loglikelihood) of the model<br>

See the nevergrad documentation regarding different optimizers<br>
https://facebookresearch.github.io/nevergrad/optimization.html#choosing-an-optimizer

In [ ]:
# TwoPointsDE is a good suggested default for some of our typical use cases
opt_class = ng.optimizers.TwoPointsDE
orunner = optimize_model(bcm, opt_class=opt_class)

In [ ]:
# You can also suggest starting points for the optimization (as well as specify an init method for unsuggested points)
# This is the "midpoint" method by default (ie the 0.5 ppf of the prior distribution)
orunner = optimize_model(bcm, opt_class=opt_class, suggested=defp, init_method="midpoint")

In [ ]:
# Some optimizers have their own initiliazation routines which will override the settings to optimize_model
# For example, the following optimizer performs a LHS search to select its initial points
# opt_class = ng.families.DifferentialEvolution(initialization="LHS", crossover="twopoints")

In [ ]:
# Here we run the optimizer in a loop, inspecting the current best point at each iteration
# Using the loss information at each step can provide the basis for stopping conditions

for i in range(8):
    # Run the minimizer for a specified 'budget' (ie number of evaluations)
    rec = orunner.minimize(100)
    # Print the loss (objective function value) of the current recommended parameters
    print(rec.loss)

In [ ]:
mle_params = rec.value[1]
mle_params

In [ ]:
# Run the model against the parameter estimates
res = bcm.run(mle_params)

In [ ]:
target = "incidence"

# You can access the targets from the BCM
bcm.targets[target].data.plot(style='.')
res.derived_outputs[target].plot()

In [ ]:
# Inspect the bias of the resulting output

diff = (res.derived_outputs[target] - bcm.targets[target].data)
print(diff.mean() / diff.std())
diff.plot();

# That's it!

Try experimenting with different optimizers, and running with your own models,
or use the MLE outputs as a starting point for further optimizations